<h3>Problem Statement:</h3> <h5> Using publicly available data on the national factors that impact the supply and demand of homes in the US, build a data science model to study the effect of these variables on home prices.</h5>

<h3>INSIGHTS</h3>: The following features are taken for consideration:

    1- Unemployment Rate<br>
    2- Per capita GDP<br>
    3- Median Household Income<br>
    4- Construction Prices<br>
    5- Consumer price index<br>
    6- Interest Rates<br>
    7- The number of new houses supplied<br>
    8- Working Population<br>
    9- Number of Households<br>
    10- Population Growth
<h5>As a proxy for home prices, the S&P Case-Shiller Index is used.</h5>

<h6>So after deciding the features I have downloaded the dataset of those feature from the following websites</h6>

 https://fred.stlouisfed.org/series/CUUR0000SEHA<br>
 https://www.census.gov/construction/nrc/<br>
 https://www.bls.gov/<br>
 https://www.bls.gov/charts/employment-situation/civilian-unemployment-rate.htm<br>
 https://www.zillow.com/research/data/

## Data Collection and Data Preprocessing

In [26]:
#importing libraries
import pandas as pd
import numpy as np

In [2]:
#Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("c1.csv")

#Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

#Selecting data post JUNE 2001
mask = df_CS["DATE"] >= "2002-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print(df_CS.shape)
df_CS.tail()

(255, 4)


,DATE,CSUSHPISA,Year,Month
250,2023-05-01,302.566,2023,5
251,2023-06-01,304.593,2023,6
252,2023-07-01,306.767,2023,7
253,2023-08-01,309.155,2023,8
254,2023-09-01,311.175,2023,9


In [28]:
df_CS.head()

,DATE,CSUSHPISA,Year,Month
0,2002-01-01,117.144,2002,1
1,2002-02-01,117.845,2002,2
2,2002-03-01,118.687,2002,3
3,2002-04-01,119.611,2002,4
4,2002-05-01,120.724,2002,5


In [3]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("Unemployment rate.csv")
mask = df_unemp["DATE"] >= "2002-07-01"
df_unemp = df_unemp[mask]

#Resetting Index
df_unemp.reset_index(inplace = True)
df_unemp.drop(columns = ["index"], inplace = True)
df_unemp.drop([255], inplace = True)
print(df_unemp.shape)
df_unemp.tail()

(255, 2)


,DATE,UNRATE
250,2023-05-01,3.7
251,2023-06-01,3.6
252,2023-07-01,3.5
253,2023-08-01,3.8
254,2023-09-01,3.8


In [4]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("GDP.csv", names = ["DATE", "Per_Capita_GDP"], skiprows = 1)
mask=df_pcgdp["DATE"] >= "2002-07-01"
df_pcgdp = df_pcgdp[mask]
print(df_pcgdp.shape)
df_pcgdp.tail()

(85, 2)


,DATE,Per_Capita_GDP
90,2022-07-01,65462.0
91,2022-10-01,65783.0
92,2023-01-01,66078.0
93,2023-04-01,66341.0
94,2023-07-01,67083.0


In [5]:

# Interest Rate Data
df_Fed_rate = pd.read_csv("interest rates.csv",names=["DATE","Interest_rate"],skiprows=1)
mask =df_Fed_rate["DATE"] >= "2002-07-01"
df_Fed_rate = df_Fed_rate[mask]
print(df_Fed_rate.shape)
df_Fed_rate.tail()

(230, 2)


,DATE,Interest_rate
255,2021-04-01,0.25
256,2021-05-01,0.25
257,2021-06-01,0.25
258,2021-07-01,0.25
259,2021-08-01,0.25


In [6]:
# Reading construction price Data into a dataframe
df_cons_price_index = pd.read_csv("construction price.csv", names = ["DATE", "Cons_Materials"], skiprows = 1)
df_cons_price_index = df_cons_price_index[df_cons_price_index['DATE'] >= '2002-07-01'] 
df_cons_price_index = df_cons_price_index[df_cons_price_index['DATE'] <= '2023-09-01'] 
print(df_cons_price_index.shape)
df_cons_price_index.tail()

(255, 2)


,DATE,Cons_Materials
280,2023-05-01,337.473
281,2023-06-01,337.336
282,2023-07-01,334.576
283,2023-08-01,333.980
284,2023-09-01,332.224


In [7]:
# Consumer Price Index
df_CPI = pd.read_csv("cpi.csv", names = ["DATE", "CPI"], skiprows = 1)
df_CPI = df_CPI[df_CPI['DATE'] >= '2002-07-01']
df_CPI = df_CPI[df_CPI['DATE'] <= '2023-09-01']
print(df_CPI.shape)
df_CPI.tail()

(255, 2)


,DATE,CPI
280,2023-05-01,303.294
281,2023-06-01,303.841
282,2023-07-01,304.348
283,2023-08-01,306.269
284,2023-09-01,307.481


In [8]:
# Monthly new house supply
df_house = pd.read_csv("Number of houses.csv", names = ["DATE", "Houses"], skiprows = 1)
df_house = df_house[df_house['DATE'] >= '2002-07-01']
df_house = df_house[df_house['DATE'] <= '2023-09-01']
print(df_house.shape)
df_house.tail()

(255, 2)


,DATE,Houses
268,2023-05-01,7.2
269,2023-06-01,7.5
270,2023-07-01,7.1
271,2023-08-01,7.8
272,2023-09-01,7.2


In [9]:
# Housing Subsidies

df_subsidy = pd.read_csv("government Subsidies.csv", names = ["DATE", "Subsidy"], skiprows = 1)
df_subsidy = df_subsidy[df_subsidy['DATE'] >= '2002-07-01']
df_subsidy = df_subsidy[df_subsidy['DATE'] <= '2023-09-01']
print(df_subsidy.shape)
df_subsidy.tail()

(20, 2)


,DATE,Subsidy
18,2018-01-01,38.859
19,2019-01-01,40.185
20,2020-01-01,44.147
21,2021-01-01,45.299
22,2022-01-01,48.021


In [38]:
df_subsidy.columns

Index(['DATE', 'Subsidy'], dtype='object')

In [10]:
# Working age population

df_working = pd.read_csv("workingagepopulation.csv", names = ["DATE", "Working_Population"], skiprows = 1)
df_working = df_working[df_working['DATE'] >= '2002-07-01']
df_working = df_working[df_working['DATE'] <= '2023-09-01']

print(df_working.shape)
df_working.tail()

(255, 2)


,DATE,Working_Population
280,2023-05-01,2.086128e+08
281,2023-06-01,2.087069e+08
282,2023-07-01,2.087792e+08
283,2023-08-01,2.089066e+08
284,2023-09-01,2.091172e+08


In [11]:
# Real Median Household Income

df_income = pd.read_csv("Median household income.csv", names = ["DATE", "Income"], skiprows = 1)
df_income = df_income[df_income['DATE'] >= '2002-07-01']
df_income = df_income[df_income['DATE'] <= '2023-09-01']
print(df_income.shape)
df_income.tail()

(20, 2)


,DATE,Income
18,2018-01-01,73030
19,2019-01-01,78250
20,2020-01-01,76660
21,2021-01-01,76330
22,2022-01-01,74580


In [12]:
# Number of households

df_households = pd.read_csv("Number of houses.csv", names = ["DATE", "Num_Households"], skiprows = 1)
df_households = df_households[df_households['DATE'] >= '2002-07-01']
df_households = df_households[df_households['DATE'] <= '2023-09-01']
print(df_households.shape)
df_households.tail()

(255, 2)


,DATE,Num_Households
268,2023-05-01,7.2
269,2023-06-01,7.5
270,2023-07-01,7.1
271,2023-08-01,7.8
272,2023-09-01,7.2


In [13]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.shape

(255, 5)

In [14]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["DATE"] = pd.to_datetime(df1["DATE"])
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()

(255, 10)


,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,Interest_rate
DATE,,,,,,,,,,
2002-07-01,122.888,2002,7,50362.0,1.838869e+08,4.2,180.0,5.8,144.7,1.25
2002-08-01,123.830,2002,8,NaN,1.841596e+08,4.0,180.5,5.7,145.1,1.25
2002-09-01,124.780,2002,9,NaN,1.844224e+08,3.9,180.8,5.7,145.4,1.25
2002-10-01,125.735,2002,10,50302.0,1.845542e+08,4.0,181.2,5.7,145.0,1.25
2002-11-01,126.670,2002,11,NaN,1.847447e+08,4.0,181.5,5.9,144.6,0.83


In [15]:
# Merging other dataframes 
others = [  df_income, df_subsidy]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()
df.shape

(255, 12)

In [29]:
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,Interest_rate,Income,Subsidy,popgrowth
6,128.461,2003,1,50462.000000,1.856353e+08,4.0,182.6,5.8,144.4,2.25,65860.0,25.93,0.859482
7,129.355,2003,2,50573.333333,1.858697e+08,4.5,183.6,5.9,145.2,2.25,65860.0,25.93,0.859482
8,130.148,2003,3,50684.666667,1.860851e+08,4.1,183.9,5.9,145.2,2.25,65860.0,25.93,0.859482
9,130.884,2003,4,50796.000000,1.864708e+08,4.1,183.2,6.0,145.9,2.25,65860.0,25.93,0.859482
10,131.735,2003,5,51034.666667,1.866491e+08,3.9,182.9,6.1,145.8,2.25,65860.0,25.93,0.859482


In [16]:
popgrowth=pd.read_csv(r"population growth rate.csv",names=["DATE","popgrowth"],skiprows=1)
popgrowth['DATE'] = pd.to_datetime(popgrowth['DATE'])
popgrowth = popgrowth[(popgrowth['DATE'] >= '2002-07-01') & (popgrowth['DATE'] <= '2023-09-01')]
popgrowth.tail()

,DATE,popgrowth
18,2018-01-01,0.526435
19,2019-01-01,0.455381
20,2020-01-01,0.964348
21,2021-01-01,0.156747
22,2022-01-01,0.377565


In [17]:
popgrowth["Year"] = pd.DatetimeIndex(popgrowth["DATE"]).year
popgrowth.set_index("DATE", inplace = True)
df = pd.merge(df,popgrowth , how = "left", on = "Year")

In [18]:
df.shape

(255, 13)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255 entries, 0 to 254
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CSUSHPISA           255 non-null    float64
 1   Year                255 non-null    int64  
 2   Month               255 non-null    int64  
 3   Per_Capita_GDP      85 non-null     float64
 4   Working_Population  255 non-null    float64
 5   Houses              255 non-null    float64
 6   CPI                 255 non-null    float64
 7   UNRATE              255 non-null    float64
 8   Cons_Materials      255 non-null    float64
 9   Interest_rate       230 non-null    float64
 10  Income              240 non-null    float64
 11  Subsidy             240 non-null    float64
 12  popgrowth           240 non-null    float64
dtypes: float64(11), int64(2)
memory usage: 27.9 KB


In [78]:
df.tail()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,Interest_rate,Income_x,Subsidy_x,Income_y,Subsidy_y,popgrowth
250,302.566,2023,5,NaN,2.086128e+08,7.2,303.294,3.7,337.473,NaN,NaN,NaN,NaN,NaN,NaN
251,304.593,2023,6,NaN,2.087069e+08,7.5,303.841,3.6,337.336,NaN,NaN,NaN,NaN,NaN,NaN
252,306.767,2023,7,67083.0,2.087792e+08,7.1,304.348,3.5,334.576,NaN,NaN,NaN,NaN,NaN,NaN
253,309.155,2023,8,NaN,2.089066e+08,7.8,306.269,3.8,333.980,NaN,NaN,NaN,NaN,NaN,NaN
254,311.175,2023,9,NaN,2.091172e+08,7.2,307.481,3.8,332.224,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.isnull().sum()

CSUSHPISA               0
Year                    0
Month                   0
Per_Capita_GDP        170
Working_Population      0
Houses                  0
CPI                     0
UNRATE                  0
Cons_Materials          0
Interest_rate          25
Income                 15
Subsidy                15
popgrowth              15
dtype: int64

The "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other columns are due to the unavailability of fresh data. We will first fill in the missing values in the "Per_Capita_GDP" column using linear interpolation. We will drop the rows with missing values in the other columns. This means that we will use data from 2002 to 2022.


In [21]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [22]:
df.dropna(inplace = True)

In [23]:
df.isnull().sum()

CSUSHPISA             0
Year                  0
Month                 0
Per_Capita_GDP        0
Working_Population    0
Houses                0
CPI                   0
UNRATE                0
Cons_Materials        0
Interest_rate         0
Income                0
Subsidy               0
popgrowth             0
dtype: int64

In [24]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

Shape of the dataframe after preprocessing:-  (224, 13)


We shall save the resultant dataframe as csv file

In [25]:
df.to_csv("resultant_dataset.csv")